In [2]:
import pandas as pd
import numpy as np

import os
import random

In [13]:
pic_file_root = './data/images/'
train_pic = "C:/Users/77369/mmdetection/data/VOCdevkit/VOC2007/ImageSets/Main/train.txt"
bbox_file = "/Users/77369/PycharmProjects/641/data/images_bboxes_withoutNaN.csv"
train_pic_df = pd.read_csv(train_pic, header=None)

total_pic = pd.read_csv(bbox_file)
total_pic = total_pic['image_id'].unique()
total_pic = [f.split('.')[0] for f in total_pic]

valid_pic = [p for p in total_pic if p not in train_pic_df.values.tolist()]
valid_pic = pd.DataFrame({'pic':valid_pic})
valid_pic.head()
show_pic = valid_pic[:50]
test_pic = valid_pic[:1000]
show_pic.to_csv('./data/test_picture/show_pic.txt', sep='\n', index=False, header=False)
test_pic.to_csv('./data/test_picture/test_pic.txt', sep='\n', index=False, header=False)

# print(len(valid_pic))

In [5]:
pic_file_root = './data/images/'
train_info = './data/Train.csv'
train_pic_df = pd.read_csv(train_info)
# train_pic_df.head()
print(train_pic_df[train_pic_df['image_id_worm'].duplicated()])

                        image_id_worm worm_type  number_of_worms
13    id_00450e8b229932359e7c32a3.jpg       pbw               33
94    id_025d41d8ecbff4e781b3d36c.jpg       abw                1
115   id_02aca55d9126de2b80f16312.jpg       pbw                7
190   id_04cc772c00e7ff5dfe712dc6.jpg       abw                1
205   id_050bfb62da75e63107eea8d6.jpg       pbw                3
...                               ...       ...              ...
9503  id_f52e952ae5d8a1bced79cdc8.jpg       abw                1
9580  id_f73ab7b153f2028ae790798c.jpg       abw                2
9615  id_f83334c80f0983276bf63c9a.jpg       abw                1
9657  id_f8f857f02c602a2850a58208.jpg       abw                3
9834  id_fe1267a85879c87751dc0233.jpg       pbw                5

[163 rows x 3 columns]


In [ ]:
show_pic_file = "./data/test_picture/show_pic.txt"
test_pic_file = "./data/test_picture/test_pic.txt"

show_pic_df = pd.read_csv(show_pic_file, header=None)
test_pic_df = pd.read_csv(test_pic_file, header=None)
show_pic_df.head()


In [10]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv

# Specify the path to model config and checkpoint file
config_file = "C:/Users/77369/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_voc0712/faster_rcnn_r50_fpn_1x_voc0712.py"
checkpoint_file = "C:/Users/77369/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_voc0712/latest.pth"

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cpu')


d:\Programming\Anaconda3\envs\EE641_Project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.1'
d:\Programming\Anaconda3\envs\EE641_Project\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


load checkpoint from local path: C:/Users/77369/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_voc0712/latest.pth


In [62]:
color = [(255, 0, 0), (0, 0, 255)]
for pic in show_pic_df.values.tolist():
    img = pic_file_root + pic[0] + '.jpg'  # or img = mmcv.imread(img), which will only load it once
    result = inference_detector(model, img)
    model.show_result(img, result, score_thr=0.7, bbox_color=color, out_file=f'./data/test_picture/show_pic_results/{pic[0]}.jpg')


In [ ]:
# 0: pwb, 1: awb

In [134]:
train_info = './data/Train.csv'
train_pic_df = pd.read_csv(train_info)
threshold = 0.7

pre_awb_total = []
pre_pwb_total = []
y_awb_total = []
y_pwb_total = []
mae_total = []
mae = 0
for pic in test_pic_df.values.tolist()[:500]:
# for pic in show_data:
    img = pic_file_root + pic[0] + '.jpg'  # or img = mmcv.imread(img), which will only load it once
    result = inference_detector(model, img)
    pre_pwb = np.sum(result[0][:, -1] >= threshold)
    pre_awb = np.sum(result[1][:, -1] >= threshold)
    true_info = train_pic_df[train_pic_df['image_id_worm'] == pic[0] + '.jpg']
    if 'pbw' in true_info['worm_type'].values:
        y_pbw = true_info[true_info['worm_type'] == 'pbw']['number_of_worms'].values[0]
    else:
        y_pbw = 0
    if 'abw' in true_info['worm_type'].values:
        y_abw = true_info[true_info['worm_type'] == 'abw']['number_of_worms'].values[0]
    else:
        y_abw = 0
    mae_temp = abs(pre_pwb - y_pbw) +abs(pre_awb - y_abw)
    mae += mae_temp

    pre_awb_total.append(pre_awb)
    pre_pwb_total.append(pre_awb)
    y_awb_total.append(y_abw)
    y_pwb_total.append(y_pbw)
    mae_total.append(mae_temp)
# mae /= len(test_pic_df.values.tolist())
# print(f"mae = {mae}")

In [137]:
for pic in test_pic_df.values.tolist()[500:]:
# for pic in show_data:
    img = pic_file_root + pic[0] + '.jpg'  # or img = mmcv.imread(img), which will only load it once
    result = inference_detector(model, img)
    pre_pwb = np.sum(result[0][:, -1] >= threshold)
    pre_awb = np.sum(result[1][:, -1] >= threshold)
    true_info = train_pic_df[train_pic_df['image_id_worm'] == pic[0] + '.jpg']
    if 'pbw' in true_info['worm_type'].values:
        y_pbw = true_info[true_info['worm_type'] == 'pbw']['number_of_worms'].values[0]
    else:
        y_pbw = 0
    if 'abw' in true_info['worm_type'].values:
        y_abw = true_info[true_info['worm_type'] == 'abw']['number_of_worms'].values[0]
    else:
        y_abw = 0
    mae_temp = abs(pre_pwb - y_pbw) +abs(pre_awb - y_abw)
    mae += mae_temp

    pre_awb_total.append(pre_awb)
    pre_pwb_total.append(pre_awb)
    y_awb_total.append(y_abw)
    y_pwb_total.append(y_pbw)
    mae_total.append(mae_temp)

In [159]:
val_data = {'image_id': [a[0] for a in test_pic_df.values.tolist()], 'pre_pwb_num': pre_pwb_total, 'pre_awb_num':pre_awb_total, 'y_pwb_num':y_pwb_total, 'y_awb_num': y_awb_total, 'mae':mae_total}
val_data = pd.DataFrame(val_data)
val_data.columns
# val_data.to_csv('./data/test_picture/val_results.csv', sep='\n', index=False)

Index(['image_id', 'pre_pwb_num', 'pre_awb_num', 'y_pwb_num', 'y_awb_num',
       'mae'],
      dtype='object')

In [144]:
show_data = ["id_8ada71185e18dc8838b4925b.jpg", "id_9eb4b51e24675e47795c7693.jpg", "id_49e44652c7aa30903dd9df34.jpg"]
color = [(255, 0, 0), (0, 0, 255)]
for pic in show_data:
    img = pic_file_root + pic  # or img = mmcv.imread(img), which will only load it once
    result = inference_detector(model, img)
    model.show_result(img, result, score_thr=0.7, bbox_color=color, out_file=f'./data/test_picture/show_pic_results/{pic}.jpg')

In [258]:
bbox_file = "/Users/77369/PycharmProjects/641/data/images_bboxes.csv"
bbox_info = pd.read_csv(bbox_file)
bbox_info.head()

,image_id,worm_type,geometry
0,id_8ada71185e18dc8838b4925b.jpg,pbw,"POLYGON ((2478.89 156.77, 2478.89 297.86, 2377..."
1,id_8ada71185e18dc8838b4925b.jpg,pbw,"POLYGON ((939.8199999999999 1221.46, 939.81999..."
2,id_8ada71185e18dc8838b4925b.jpg,abw,"POLYGON ((1559.7 1558.06, 1559.7 1966.3, 1129...."
3,id_bc86d3cfd0c0463fa5cae33e.jpg,abw,POLYGON ((2090.216399557101 1537.2100674812148...
4,id_bc86d3cfd0c0463fa5cae33e.jpg,abw,POLYGON ((2342.2568587778546 1676.304250464547...


In [190]:
img = pic_file_root + "id_8ada71185e18dc8838b4925b.jpg"  # or img = mmcv.imread(img), which will only load it once
result_temp = inference_detector(model, img)
print(result_temp[0])
print(result_temp[1])

[[7.1676733e+02 1.2134570e+03 9.3951703e+02 1.3211289e+03 9.8988241e-01]
 [2.3727783e+03 1.5602167e+02 2.4830454e+03 2.9145331e+02 8.0207944e-01]]
[[1.0998333e+03 1.5218259e+03 1.6024414e+03 1.9905519e+03 9.9776471e-01]]


In [280]:
import re
show_data_my = ["id_8ada71185e18dc8838b4925b", "id_9eb4b51e24675e47795c7693", "id_49e44652c7aa30903dd9df34", "id_0a99918f6e85f734a00d969d", "id_4780c30c76a533d0b93426e3", "id_2e8cd44287db582e0d2d449d"]
for id in show_data_my:
    i_bbox_info = bbox_info[bbox_info['image_id'] == id+'.jpg']
    # i_bbox_info = bbox_info[bbox_info['image_id'] == 'id_49e44652c7aa30903dd9df34.jpg']
    pbw_list = []
    abw_list = []
    for j in i_bbox_info.values:
        if j.shape[0] < 3:
            print(id)
            continue
        point = re.findall(r'[(](.*?)[)]', j[-1])[0][1:]
        point = point.split(',')
        for i in range(5):
            point[i] = point[i].lstrip()
            point[i] = point[i].split(' ')
            point[i] = [float(a) for a in point[i]]
        if j[-2] == 'abw':
            abw_list.append(point[0]+point[2]+[1])
        else:
            pbw_list.append(point[0]+point[2]+[1])
    point_info = [np.array(pbw_list).reshape(-1,5), np.array(abw_list).reshape(-1,5)]
    model.show_result(pic_file_root + id +'.jpg', point_info, score_thr=0.7, bbox_color=color, out_file=f'./data/test_picture/original_pic/{id}.jpg')


In [ ]:
for pic in test_pic_df.values.tolist()[500:]:
# for pic in show_data:
    img = pic_file_root + pic[0] + '.jpg'  # or img = mmcv.imread(img), which will only load it once
    result = inference_detector(model, img)
    pre_pwb = np.sum(result[0][:, -1] >= threshold)
    pre_awb = np.sum(result[1][:, -1] >= threshold)
    true_info = train_pic_df[train_pic_df['image_id_worm'] == pic[0] + '.jpg']
    if 'pbw' in true_info['worm_type'].values:
        y_pbw = true_info[true_info['worm_type'] == 'pbw']['number_of_worms'].values[0]
    else:
        y_pbw = 0
    if 'abw' in true_info['worm_type'].values:
        y_abw = true_info[true_info['worm_type'] == 'abw']['number_of_worms'].values[0]
    else:
        y_abw = 0
    mae_temp = abs(pre_pwb - y_pbw) +abs(pre_awb - y_abw)
    mae += mae_temp

    pre_awb_total.append(pre_awb)
    pre_pwb_total.append(pre_awb)
    y_awb_total.append(y_abw)
    y_pwb_total.append(y_pbw)
    mae_total.append(mae_temp)

In [9]:
test_info = './data/Test.csv'
test_pic_df = pd.read_csv(test_info)
test_pic_df.values.tolist()[0][0]

'id_00332970f80fa9a47a39516d.jpg'

In [ ]:
test_info = './data/Test.csv'
test_pic_df = pd.read_csv(test_info)
threshold = 0.7

test_pre_total = []
image_id = []
# test_pre_pwb_total = []
# [:500]
for pic in test_pic_df.values.tolist():
    img = pic_file_root + pic[0]  # or img = mmcv.imread(img), which will only load it once
    result = inference_detector(model, img)
    pre_pbw = np.sum(result[0][:, -1] >= threshold)
    pre_abw = np.sum(result[1][:, -1] >= threshold)
    image_id.append(pic[0].split('.')[0]+'_abw')
    image_id.append(pic[0].split('.')[0]+'_pbw')
    test_pre_total.append(pre_abw)
    test_pre_total.append(pre_pbw)
print(image_id.shape)
print(test_pre_total.shape)
# mae /= len(test_pic_df.values.tolist())
# print(f"mae = {mae}")

In [14]:
print(len(test_pre_total))
print(test_pic_df.shape)

5606
(2803, 1)


In [16]:
test_out = {
    'image_id_worm': image_id,
    'number_of_worms': test_pre_total
}
test_out_df = pd.DataFrame(test_out)
test_out_df.head()

,image_id_worm,number_of_worms
0,id_00332970f80fa9a47a39516d_abw,4
1,id_00332970f80fa9a47a39516d_pbw,0
2,id_0035981bc3ae42eb5b57a317_abw,0
3,id_0035981bc3ae42eb5b57a317_pbw,18
4,id_005102f664b820f778291dee_abw,9


In [18]:
test_out_df.to_csv('./data/test_result.csv', index=False)